#Set the environment

In [12]:
#mount google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Install requirements.txt.
#Given path might not work for you as your location might be different
#Find the requirements.txt and copy path and replace this path

!pip install -r /content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/requirements.txt

In [14]:
#These are all the imports necessary. If you encounter anything missing just import here (hopefully that won't be the case).

import os
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pickle
import torch
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import sys

#sys.append Basic folder and Helper folder. Both can be found in the Models subfolder. copy your path and replace it.

sys.path.append('/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic')
sys.path.append('/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Helper')

import load_feature
import helper

In [15]:
#Change our directory where n-gram.py is located.
#Given path might not work for you as your location might be different
#Find the "Basic" sub-folder in the "Models" folder where n-gram.py is located and replace this path

%cd drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic

[Errno 2] No such file or directory: 'drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic'
/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic


#Train model

In [16]:
#Run our model
#Change exp_name and model_name according to your work
#For example : " !python n-gram.py Unigram SVM -s ""
#Refer to https://github.com/Rowan1697/FakeNews for all the available experiment names and models

!python n-gram.py Bigram LR -s

#After your model has been trained you can find the saved RESULT in the results folder
#After your model has been trained you can find the saved MODEL in the saved-model folder
#Your saved model will have the naming convenction of "model name" + '_' + "experiment name" + ".pkl"
#For example "LR_C3-gram.pkl"

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['পাওয়া', 'পেয়ে', 'বিষয়টি', 'হইয়া', 'হয়েছিল'] not in stop_words.
  warnings.warn(
                                Weighted Avg         #            Non-Violence           #          Passive Violence         #          Active Violence          
                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score
Bigram             0.58         0.54        0.49      #  0.50         0.90         0.65      #  0.62         0.23         0.33      #  0.68         0.30         0.42
                                Macro 

#Testing

In [17]:
#set the path of our model and datasets
#You can find the models in saved models folder under Basic
#You can find the datasets in dataset folder under 4-2

model_path = '/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic/Saved Models/LR_Bigram.pkl'
train_data_path = '/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/Datasets/train.csv'
test_data_path = '/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/Datasets/test.csv'

In [18]:
#Prepare the test dataset

df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

# Rename columns to match what load_feature.py expects
# Your columns: 'Text' → rename to 'Content'
#               'Labels' → rename to 'Label' (singular, as used in word_emb)
df_train = df_train.rename(columns={'text': 'Content', 'label': 'Label'})
df_test = df_test.rename(columns={'text': 'Content', 'label': 'Label'})

# Now create X_train and X_test as DataFrames with 'Content' column
X_train = df_train[['Content']]
X_test = df_test[['Content']]

y_test = df_test['Label'].values.ravel()

#For Unigram set a = 1, b = 1
#For Bigram set a = 2, b = 2
#For Trigram set a = 3, b = 3
#For U+B+T set a = 1, b = 3

#For C3-gram set a = 3, b = 3
#For C4-gram set a = 4, b = 4
#For C5-gram set a = 5, b = 5
#For C3+C4+C5 set a = 3, b = 5

a = 2
b = 2

#If you are using U/B/T : use wordF
#If you are using C3/C4/C5 : use charF
#For example when using C3, I have only used charF, and commented out wordF.

X_test = load_feature.tfidf_wordF(X_train, X_test, a, b)
# X_test = load_feature.tfidf_charF(X_train, X_test, a, b)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['পাওয়া', 'পেয়ে', 'বিষয়টি', 'হইয়া', 'হয়েছিল'] not in stop_words.
  warnings.warn(


In [19]:
#load and Test the model

clf = pickle.load(open(model_path, 'rb'))
y_pred = clf.predict(X_test)

In [20]:
#load and Test the model

clf = pickle.load(open(model_path, 'rb'))
y_pred = clf.predict(X_test)

class0, class1, class2, o_weighted, o_macro = helper.getResult(y_test, y_pred)

print("                                Weighted Avg         #            Non-Violence           #          Passive Violence         #          Active Violence          ")
print("                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score")

#Set the 'model' and 'experiment' names
model = 'LR'
experiment = 'Bigram'
exp = model+'_'+experiment

res = helper.printResult(exp, o_weighted, class0, class1, class2)
print(res)

path = model+"_final_results.txt"
helper.saveResults(path, res)

print("                                Macro Avg            #            Non-Violence           #          Passive Violence         #          Active Violence          ")
print("                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score")

res = helper.printResult(exp, o_macro, class0, class1, class2)
print(res)

path = model+"_final_results_macro.txt"
helper.saveResults(path, res)

                                Weighted Avg         #            Non-Violence           #          Passive Violence         #          Active Violence          
                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score
LR_Bigram          0.58         0.53        0.48      #  0.49         0.88         0.63      #  0.60         0.24         0.34      #  0.72         0.29         0.41
                                Macro Avg            #            Non-Violence           #          Passive Violence         #          Active Violence          
                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score
LR_Bigram          0.61         0.47        0.46      #  0.49         0.88         0.63      #  0.60         0.24         0.34      #  0.72         0.29         0.41
